## Note: this doesn't work super reliably yet, mainly because the CAN messages for pos/vel gains aren't updated in ODrive firmware.

In [ ]:
%load_ext autoreload
%autoreload 1
%aimport communicate
from communicate import CableRobot, MotorState, ControllerState
import numpy as np
import time

In [ ]:
def configure_anticogging(axis):
    with CableRobot(print_raw=True, silent=False, write_timeout=None, port='/dev/tty.usbmodem100994303') as robot:
        robot.update()
        robot.send(f'c{axis}')  # calibrate
        robot.silent = True
        print(robot.log_data['controller'])
        print('Waiting to begin calibration...')
        while robot.log_data['motors'][axis] is None or robot.log_data['motors'][axis].state <= 1: # wait for calibration to begin
            robot.update()
        print('Waiting to finish calibration...')
        while robot.log_data['motors'][axis].state != 1: # wait for calibration to finish
            robot.update()
        print('Starting anticogging calibration!')
        robot.silent = False
        robot.send(f'{axis}n27c0.2,5.0')  # set vel gains
        robot.send(f'{axis}n26c50.0')  # set pos gain
        robot.send(f'{axis}n11c3,1')  # set controller mode to position, passthrough
        robot.send(f'{axis}n7c8')  # start closed loop control
        robot.send(f'{axis}n16c')  # start anticogging
        robot.update()
        time.sleep(0.5)
        robot.update()
def set_gains(axis, vel_gain, vel_integrator_gain, pos_gain):
    with CableRobot(print_raw=True, silent=False, write_timeout=None, port='/dev/tty.usbmodem100994303') as robot:
        robot.send(f'{axis}n26c{pos_gain}')  # set pos gain
        robot.send(f'{axis}n27c{vel_gain},{vel_integrator_gain}')  # set vel gains
        robot.update()
        time.sleep(0.1)
        robot.update()
def enable_all():
    with CableRobot(print_raw=True, silent=False, write_timeout=None, port='/dev/tty.usbmodem100994303') as robot:
        for i in range(4):
            robot.send(f'{i}n7c8')  # start closed loop control
        robot.update()

In [ ]:
configure_anticogging(0)
configure_anticogging(1)

In [ ]:
enable_all()

In [ ]:
set_gains(1, 0.5, 5.0, 5.0)